In [1]:
import tensorflow as tf
import numpy as np
import nengo_dl
import nengo
import random
import _init_paths

from utils.base_utils.exp_utils import get_grouped_slices_2d_pooling
from utils.nengo_dl_utils import get_max_pool_global_net
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)

# Change `channels_last` to `channels_first`

In [2]:
print("Current Image Data Format: ", tf.keras.backend.image_data_format())
#tf.keras.backend.set_image_data_format("channels_first") -> For some reason this automatic change doesn't work.
#print("Changed Image Data Format to: ", tf.keras.backend.image_data_format())

Current Image Data Format:  channels_last


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# Make Channels First image coding.
train_images, test_images = np.expand_dims(train_images, -1), np.expand_dims(test_images, -1)
train_images, test_images = np.moveaxis(train_images, -1, 1), np.moveaxis(test_images, -1, 1)
print(train_images.shape, test_images.shape) # Channels First coding.

(60000, 1, 28, 28) (10000, 1, 28, 28)


# TF Model Definition

In [4]:
# input
inp = tf.keras.Input(shape=(1, 28, 28)) # Channels First

# convolutional layers
conv0 = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation=tf.nn.relu,
    data_format="channels_first"
)(inp)

# Default pool_size = (2,2), padding = "valid", data_format = "channels_first" (changed programmatically above).
max_pool0 = tf.keras.layers.MaxPool2D(data_format="channels_first")(conv0) 

conv1 = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    activation=tf.nn.relu,
    data_format="channels_first"
)(max_pool0)

# max_pool1 = tf.keras.layers.MaxPool2D(data_format="channels_first")(conv1) 

# conv2 = tf.keras.layers.Conv2D(
#     filters=64,
#     kernel_size=3,
#     strides=2,
#     activation=tf.nn.relu,
#     data_format="channels_first"
# )(max_pool1)


# fully connected layer
flatten = tf.keras.layers.Flatten()(conv1)
dense = tf.keras.layers.Dense(units=10, activation="softmax")(flatten)

model = tf.keras.Model(inputs=inp, outputs=dense)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 28, 28)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 26, 26)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 13, 13)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 6, 6)          18496     
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                23050     
Total params: 41,866
Trainable params: 41,866
Non-trainable params: 0
_________________________________________________________

In [26]:
print(conv0.name, max_pool0.name, conv1.name)

conv2d/Identity:0 max_pooling2d/Identity:0 conv2d_1/Identity:0


## TF Model Compilation and Evaluation

In [6]:
model.compile(
  optimizer=tf.optimizers.Adam(0.001),
  loss=tf.losses.SparseCategoricalCrossentropy(),
  metrics=[tf.metrics.sparse_categorical_accuracy])
model.fit(train_images, train_labels, epochs=4)

Epoch 1/4
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2590 - sparse_categorical_accuracy: 0.9478
Epoch 2/4
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0705 - sparse_categorical_accuracy: 0.9789
Epoch 3/4
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0574 - sparse_categorical_accuracy: 0.9824
Epoch 4/4
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0473 - sparse_categorical_accuracy: 0.9855


In [7]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 2s 5ms/step - loss: 0.0695 - sparse_categorical_accuracy: 0.9815


[0.0694713443517685, 0.9815000295639038]

# Check Nengo DL MaxPool

In [8]:
n_steps, sfr = 40, 100
radius = 1000/sfr

np.random.seed(100)
ndl_model_1 = nengo_dl.Converter(model, 
                               swap_activations={tf.nn.relu: nengo.SpikingRectifiedLinear()},
                               scale_firing_rates=sfr,
                               synapse=0.005)

with ndl_model_1.net:
  nengo_dl.configure_settings(stateful=False)

/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:588: UserWarning: Activation type <function softmax at 0x2b5c422c6cb0> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation


## Explicitly add the synapse between the Conv layer and the MaxPool layer (NengoDL existing bug)

In [9]:
# BUG: https://github.com/nengo/nengo-dl/issues/214

print("Before:", ndl_model_1.net._connections[3], ndl_model_1.net._connections[3].synapse)
ndl_model_1.net._connections[3].synapse = nengo.Lowpass(0.005)
print("After:", ndl_model_1.net._connections[3], ndl_model_1.net._connections[3].synapse)

# print("Before:", ndl_model_1.net._connections[7], ndl_model_1.net._connections[7].synapse)
# ndl_model_1.net._connections[7].synapse = nengo.Lowpass(0.005)
# print("After:", ndl_model_1.net._connections[7], ndl_model_1.net._connections[7].synapse)

Before: <Connection from <Neurons of <Ensemble "conv2d.0">> to <TensorNode "max_pooling2d">> None
After: <Connection from <Neurons of <Ensemble "conv2d.0">> to <TensorNode "max_pooling2d">> Lowpass(tau=0.005)


In [10]:
ndl_model_1.net._connections

[<Connection at 0x2b5ca40fe5d0 from <Node "conv2d.0.bias"> to <Node "conv2d.0.bias_relay">>,
 <Connection at 0x2b5cbe1c9d10 from <Node "conv2d.0.bias_relay"> to <Neurons of <Ensemble "conv2d.0">>>,
 <Connection at 0x2b5cbff1e890 from <Node "input_1"> to <Neurons of <Ensemble "conv2d.0">>>,
 <Connection at 0x2b5cbef81a10 from <Neurons of <Ensemble "conv2d.0">> to <TensorNode "max_pooling2d">>,
 <Connection at 0x2b5cbe2fec50 from <Node "conv2d_1.0.bias"> to <Node "conv2d_1.0.bias_relay">>,
 <Connection at 0x2b5cbe2fead0 from <Node "conv2d_1.0.bias_relay"> to <Neurons of <Ensemble "conv2d_1.0">>>,
 <Connection at 0x2b5cfae35150 from <TensorNode "max_pooling2d"> to <Neurons of <Ensemble "conv2d_1.0">>>,
 <Connection at 0x2b5cfae35390 from <Node "dense.0.bias"> to <TensorNode "dense.0">>,
 <Connection at 0x2b5cfae352d0 from <Neurons of <Ensemble "conv2d_1.0">> to <TensorNode "dense.0">>]

In [11]:
ndl_test_images = np.tile(
  test_images.reshape((test_images.shape[0], 1, -1)), (1, n_steps, 1))
ndl_input_1 = ndl_model_1.inputs[inp]
ndl_output_1 = ndl_model_1.outputs[dense]

In [12]:
with nengo_dl.Simulator(
  ndl_model_1.net, minibatch_size=100) as sim:
  data1 = sim.predict({ndl_input_1: ndl_test_images[:200]})

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


In [13]:
acc = 0
for pred, true in zip(data1[ndl_output_1][:, -1, :], test_labels):
  if np.argmax(pred) == true:
    acc += 1
print(acc/200)

0.995


# Replace the Nengo-DL MaxPool TensorNode with the custom Max Pool Network

In [14]:
# Get the grouped slices for 2x2 MaxPooling.
grouped_slices_1 = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=32, rows=26, cols=26)
conn_from_conv0_to_max_pool = ndl_model_1.net.all_connections[3]
conn_from_max_pool_to_conv1 = ndl_model_1.net.all_connections[6]

## Connect the max_pool_layer1 to the prev and next Conv (Ensemble).


In [15]:
with ndl_model_1.net:   
  # Get the MaxPool Global Network.
  max_pool_layer_1 = get_max_pool_global_net((32, 26, 26), radius=3, sf=1.2) # 32 x 26 x 26 = 21632 from the prev Conv layer.
  
  # Connection from Conv0 to max_pool_layer_1.
  nengo.Connection(
      conn_from_conv0_to_max_pool.pre_obj[grouped_slices_1], #These are of type nengo.ensemble.Neurons
      max_pool_layer_1.input,
      transform=conn_from_conv0_to_max_pool.transform,
      synapse=conn_from_conv0_to_max_pool.synapse,
      function=conn_from_conv0_to_max_pool.function
      )

  # Connection from max_pool_layer_1 to Conv1.
  nengo.Connection(
      max_pool_layer_1.output,
      conn_from_max_pool_to_conv1.post_obj, # These are of type nengo.ensemble.Neurons
      transform=conn_from_max_pool_to_conv1.transform,
      synapse=conn_from_max_pool_to_conv1.synapse,
      function=conn_from_max_pool_to_conv1.function
      )
  
  # Remove old connections.
  ndl_model_1.net._connections.remove(conn_from_conv0_to_max_pool)
  ndl_model_1.net._connections.remove(conn_from_max_pool_to_conv1)
  ndl_model_1.net._nodes.remove(conn_from_conv0_to_max_pool.post_obj)

## Check the new connections

In [16]:
ndl_model_1.net._connections

[<Connection at 0x2b5ca40fe5d0 from <Node "conv2d.0.bias"> to <Node "conv2d.0.bias_relay">>,
 <Connection at 0x2b5cbe1c9d10 from <Node "conv2d.0.bias_relay"> to <Neurons of <Ensemble "conv2d.0">>>,
 <Connection at 0x2b5cbff1e890 from <Node "input_1"> to <Neurons of <Ensemble "conv2d.0">>>,
 <Connection at 0x2b5cbe2fec50 from <Node "conv2d_1.0.bias"> to <Node "conv2d_1.0.bias_relay">>,
 <Connection at 0x2b5cbe2fead0 from <Node "conv2d_1.0.bias_relay"> to <Neurons of <Ensemble "conv2d_1.0">>>,
 <Connection at 0x2b5cfae35390 from <Node "dense.0.bias"> to <TensorNode "dense.0">>,
 <Connection at 0x2b5cfae352d0 from <Neurons of <Ensemble "conv2d_1.0">> to <TensorNode "dense.0">>,
 <Connection at 0x2b5d9c068c10 from <Neurons of <Ensemble "conv2d.0">>[[    0     1    26 ... 21605 21630 21631]] to <Node (unlabeled) at 0x2b5cfb65f450>>,
 <Connection at 0x2b5d9c068c50 from <Node (unlabeled) at 0x2b5cfb65f390> to <Neurons of <Ensemble "conv2d_1.0">>>]

# Check the Nengo-DL model with Custom MaxPooling layer

In [17]:
with nengo_dl.Simulator(
  ndl_model_1.net, minibatch_size=100) as sim:
  data2 = sim.predict({ndl_input_1: ndl_test_images[:200]})

Build finished in 0:02:06                                                      
|                     #    Optimizing graph                           | 0:04:47


KeyboardInterrupt: 

In [18]:
acc = 0
for pred, true in zip(data2[ndl_output_1][:, -1, :], test_labels):
  if np.argmax(pred) == true:
    acc += 1
print(acc/200)

0.99
